In [77]:
# dependencies
import pymongo
import nibabel as nib
from os.path import join, exists, split
from dipy.tracking import utils
import os
import numpy as np
from dipy.tracking.utils import subsegment

In [78]:
# pull painter objects from a specific freesurfer ID

def get_collection(port=3001):
    from pymongo import MongoClient
    client = MongoClient("localhost", port)
    db =  client.meteor
    collection = db.subjects
    return collection, client

collection, client = get_collection(5051)

In [12]:
# this lists all the entries associated with a specific subject id
# you can use this to see what data is there. you'll need "entry_type:lst" as that has the painters
# you'll also need "entry_type:freesurfer" for

cursor = collection.find({"subject_id":"mse65"})
for item in cursor:
    print()
    print(item)
    print()


{'subject_id': 'mse65', '_id': ObjectId('57a279df142c116598dfadfe'), 'check_masks': ['mse65/nii/ms534-mse65-001-SCOUTS_132Lbs_WEIGHT-000.nii.gz'], 'entry_type': 'nifti', 'name': 'ms534-mse65-001-SCOUTS_132Lbs_WEIGHT-000'}


{'subject_id': 'mse65', '_id': ObjectId('57a279df142c116598dfadff'), 'check_masks': ['mse65/nii/ms534-mse65-001-SCOUTS_132Lbs_WEIGHT-001.nii.gz'], 'entry_type': 'nifti', 'name': 'ms534-mse65-001-SCOUTS_132Lbs_WEIGHT-001'}


{'subject_id': 'mse65', '_id': ObjectId('57a279df142c116598dfae00'), 'check_masks': ['mse65/nii/ms534-mse65-001-SCOUTS_132Lbs_WEIGHT-002.nii.gz'], 'entry_type': 'nifti', 'name': 'ms534-mse65-001-SCOUTS_132Lbs_WEIGHT-002'}


{'subject_id': 'mse65', '_id': ObjectId('57a279df142c116598dfae01'), 'check_masks': ['mse65/nii/ms534-mse65-002-AX_T1_3D_IRSPGR_te_1.6.nii.gz'], 'entry_type': 'nifti', 'name': 'ms534-mse65-002-AX_T1_3D_IRSPGR_te_1.6'}


{'subject_id': 'mse65', '_id': ObjectId('57a279df142c116598dfae02'), 'check_masks': ['mse65/nii/ms534-mse65

In [25]:
f = open("/Users/amitvakula/Documents/test.txt",'r')
f.read()

'hello\n'

In [26]:
cursor = collection.find({"subject_id":"mse65", "entry_type":"freesurfer"})
results = []
for item in cursor:
    results.append(item)
assert len(results) == 1, "more than one result! either modify the code or restrict your query Amit"
subject = results[0]
subject["check_masks"]

DATA_HOME = "/data/henry7/PBR/subjects/"

segmentation_mask = nib.load(DATA_HOME + subject["check_masks"][1])

In [13]:
cursor = collection.find({"subject_id":"mse65", "entry_type":"lst"})
results = []
for item in cursor:
    results.append(item)
assert len(results) == 1, "more than one result! either modify the code or restrict your query Amit"
painter_entry = results[0]

In [21]:
print(len(painter_entry['painters']))
painter_entry['painters'] = painter_entry['painters'][0:1]


13


In [32]:
# ABOVE: clean up function and have it take in a subject ID and return (1) painter objects and (2) paths to data
# extract data to get the segmentation masks
# NOTE: where the data lives: /data/henry7/PBR/subjects/
painter = painter_entry['painters'][0]
painter

{'checkedBy': 'ssacco',
 'matrix_coor': [{'old_val': 28, 'x': 159, 'y': 162, 'z': 84},
  {'old_val': 28, 'x': 160, 'y': 162, 'z': 84},
  {'old_val': 28, 'x': 160, 'y': 163, 'z': 84},
  {'old_val': 28, 'x': 160, 'y': 164, 'z': 84},
  {'old_val': 28, 'x': 159, 'y': 161, 'z': 84},
  {'old_val': 28, 'x': 159, 'y': 160, 'z': 84},
  {'old_val': 28, 'x': 160, 'y': 160, 'z': 84},
  {'old_val': 28, 'x': 160, 'y': 161, 'z': 84},
  {'old_val': 28, 'x': 160, 'y': 158, 'z': 84},
  {'old_val': 28, 'x': 160, 'y': 157, 'z': 84},
  {'old_val': 28, 'x': 159, 'y': 159, 'z': 84}],
 'offset': [],
 'paintValue': 0,
 'start_point': {'x': 667, 'y': 680},
 'uuid': 'a86f26a7-11fb-0aa7-b499-b225e3521a3a',
 'world_coor': [{'x': 27.190002441406254,
   'y': -4.417007446289052,
   'z': 12.207099914550781},
  {'x': 28.127502441406254, 'y': -4.417007446289052, 'z': 12.207099914550781},
  {'x': 28.127502441406254, 'y': -5.354507446289052, 'z': 12.207099914550781},
  {'x': 28.127502441406254, 'y': -6.292007446289052, 'z

In [28]:
def get_segmentation_mask(db, subject_id="mse65"):
    cursor = db.find({"subject_id":subject_id, "entry_type":"freesurfer"})
    results = []
    for item in cursor:
        results.append(item)
    assert len(results) == 1, "more than one result! either modify the code or restrict your query Amit"
    subject = results[0]
    DATA_HOME = "/data/henry7/PBR/subjects/"
    subject["check_masks"][1]
    return nib.load(DATA_HOME + subject["check_masks"][1])

seg_mask = get_segmentation_mask(collection, "mse65")

In [29]:
# affine transformations
def get_papaya_aff(img):
    vs = img.header.get_zooms()
    aff = img.get_affine()
    ort = nib.orientations.io_orientation(aff)
    papaya_aff = np.zeros((4, 4))
    for i, line in enumerate(ort):
        papaya_aff[line[0],i] = vs[i]*line[1]
    papaya_aff[:, 3] = aff[:, 3]
    return papaya_aff

In [59]:
# convert the painter object points into the correct space

def convert_to_indices(streamline, papaya_aff, aff, img):
    #print(streamline)
    topoints = lambda x : np.array([[m["x"], m["y"], m["z"]] for m in x["world_coor"]])
    points_orig = topoints(streamline)
    points_nifti_space = list(utils.move_streamlines([points_orig], aff, input_space=papaya_aff))[0]
    from dipy.tracking._utils import _to_voxel_coordinates, _mapping_to_voxel
    lin_T, offset = _mapping_to_voxel(aff, None)
    idx = _to_voxel_coordinates(points_orig, lin_T, offset)
    return points_nifti_space, idx

def convert_to_volume(drawing, papaya_aff, aff, img, do_subsegment = True):

    topoints = lambda x : np.array([[m["x"], m["y"], m["z"]] for m in x["world_coor"]])
    points_orig = list(map(topoints, drawing))
    if do_subsegment:
        points = list(subsegment(points_orig, 0.5))
    else:
        points = points_orig
    mask2 = utils.density_map(points, img.shape, affine=papaya_aff)
    points_nifti_space = list(utils.move_streamlines(points, aff, input_space=papaya_aff))
    mask1 = utils.density_map(points_nifti_space, img.shape, affine=aff)

    #print((mask1 == mask2).all())
    # img1 = nib.Nifti1Image(mask1)
    #print(mask1.sum(), mask2.sum())
    
    return mask1, points_nifti_space

In [36]:
nifti_points, trans_points = convert_to_indices(painter, get_papaya_aff(seg_mask), seg_mask.affine, seg_mask)
print("nifti")
print(nifti_points)
print("trans")
print(trans_points)

nifti
[[ 27.19000244  -4.41700745  12.20709991]
 [ 28.12750244  -4.41700745  12.20709991]
 [ 28.12750244  -5.35450745  12.20709991]
 [ 28.12750244  -6.29200745  12.20709991]
 [ 27.19000244  -3.47950745  12.20709991]
 [ 27.19000244  -2.54200745  12.20709991]
 [ 28.12750244  -2.54200745  12.20709991]
 [ 28.12750244  -3.47950745  12.20709991]
 [ 28.12750244  -0.66700745  12.20709991]
 [ 28.12750244   0.27049255  12.20709991]
 [ 27.19000244  -1.60450745  12.20709991]]
trans
[[ 98 128  95]
 [ 97 128  95]
 [ 97 128  94]
 [ 97 128  93]
 [ 98 128  96]
 [ 98 128  97]
 [ 97 128  97]
 [ 97 128  96]
 [ 97 128  99]
 [ 97 128 100]
 [ 98 128  98]]


/Users/amitvakula/anaconda/envs/research3/lib/python3.5/site-packages/ipykernel/__main__.py:4: DeprecationWarning: get_affine method is deprecated.
Please use the ``img.affine`` property instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0
/Users/amitvakula/anaconda/envs/research3/lib/python3.5/site-packages/ipykernel/__main__.py:8: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [69]:
# the actual paint function (put helper functions above where I convert to the right space?)
def get_points_to_paint(drawing, papaya_affine, aff, img, outfilepath, name, authors, suffix=""):
    import pandas as pd
    df = pd.DataFrame()

    for d in drawing:
        pv = d["paintValue"]
        points_nii_space, trans_points = convert_to_indices(d, papaya_affine, aff, img)
        tmp = []
        for ni in trans_points:
            tmp.append({"x": ni[0], "y":ni[1], "z": ni[2], "val": pv})
        df = df.append(pd.DataFrame(tmp), ignore_index=True)
    df.drop_duplicates(inplace=True)
    #if not exists(join(cc["output_directory"], outfilepath)):
        #os.makedirs(join(cc["output_directory"],outfilepath))
        #print(join(cc["output_directory"], outfilepath), "created")
    #outfilename = join(outfilepath, "{}-{}{}.csv".format(name,"-".join(authors), suffix))
    #nib.Nifti1Image(mask.astype(np.float32), affine=aff).to_filename(join(cc["output_directory"], outfilename))
    #df.to_csv(join(cc["output_directory"], outfilename))
    #return outfilename
    return df

def create_paint_volume(drawing, img, output, outfilepath):
    #from pbr.config import config as cc
    #img = nib.load(join(cc["output_directory"],output["check_masks"][-1]))
    aff = img.get_affine() #affine()
    papaya_affine = get_papaya_aff(img)
    data = np.zeros(img.shape)
    outputfiles = []
    mse = output["subject_id"]
    sequence = output["name"]
    #for c in output["contours"]:
    drawing_old = output["painters"]
    #drawing = []
    #for p in drawing_old:
    #    if len(p["world_coor"]):
    #        drawing.append(p)

    name = output["entry_type"] #contour['name'].replace(" ", "_")
    authors = set([q["checkedBy"] for q in drawing])

    data = img.get_data() + data
    
    df = get_points_to_paint(drawing, papaya_affine, aff, img, outfilepath, name, authors, suffix="")
    for item in df.iterrows():
        paint_value = item[1]['val']
        x,y,z = item[1]['x'], item[1]['y'], item[1]['z']
        data[x][y][z] = paint_value
    
    painted_image = nib.nifti1.Nifti1Image(data,aff,img.header)
    
    nib.save(painted_image,os.path.join(outfilepath,"painted.nii.gz"))
    
        
    #mask, points_nifti_space = convert_to_volume(drawing, papaya_affine, aff, img, False)

    #outfilepath = join(mse, "mindcontrol/{}/{}/rois".format(sequence, output["entry_type"]))

    #papaya_aff_points = paintVolume(drawing, papaya_affine, aff, img, outfilepath, name, authors)
    #print("painter wrote", join(cc["output_directory"], outfilename))
    #outputfiles.append(outfilename)

    #aff_points = paintVolume(drawing, aff, aff, img, outfilepath, name, authors, suffix="_origAff")
    #print("painter wrote", join(cc["output_directory"], outfilename))
    #outputfiles.append(outfilename)
    
    #to_rtn = paint_over()
    
    return painted_image

In [70]:
#print(painter)
data_frame = get_points_to_paint([painter], get_papaya_aff(seg_mask), seg_mask.affine, seg_mask, None, None, None)
#print(data_frame)
points_to_write = []
for item in data_frame.iterrows():
    print(item[1]['val'])

0
0
0
0
0
0
0
0
0
0
0


/Users/amitvakula/anaconda/envs/research3/lib/python3.5/site-packages/ipykernel/__main__.py:4: DeprecationWarning: get_affine method is deprecated.
Please use the ``img.affine`` property instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0
/Users/amitvakula/anaconda/envs/research3/lib/python3.5/site-packages/ipykernel/__main__.py:8: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [71]:
modified_segmentation = create_paint_volume([painter_entry['painters'][0]], seg_mask, painter_entry, "/Users/amitvakula/Documents/research/results")

/Users/amitvakula/anaconda/envs/research3/lib/python3.5/site-packages/ipykernel/__main__.py:26: DeprecationWarning: get_affine method is deprecated.
Please use the ``img.affine`` property instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0
/Users/amitvakula/anaconda/envs/research3/lib/python3.5/site-packages/ipykernel/__main__.py:4: DeprecationWarning: get_affine method is deprecated.
Please use the ``img.affine`` property instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0
/Users/amitvakula/anaconda/envs/research3/lib/python3.5/site-packages/ipykernel/__main__.py:8: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [79]:
diff_map = seg_mask.get_data() - modified_segmentation.get_data()
for i in range(np.shape(diff_map)[0]):
    for j in range(np.shape(diff_map)[1]):
        for k in range(np.shape(diff_map)[2]):
            if diff_map[i][j][k] != 0:
                print("at {} we got value {}".format((i,j,k),diff_map[i][j][k]))

at (97, 128, 93) we got value 43.0
at (97, 128, 94) we got value 43.0
at (97, 128, 95) we got value 43.0
at (97, 128, 96) we got value 43.0
at (97, 128, 97) we got value 43.0
at (97, 128, 99) we got value 43.0
at (97, 128, 100) we got value 43.0
at (98, 128, 95) we got value 43.0
at (98, 128, 96) we got value 43.0
at (98, 128, 97) we got value 43.0
at (98, 128, 98) we got value 43.0
